In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.layers import Input, Conv1D, GlobalAvgPool2D, GlobalAveragePooling1D, \
    Dropout, Dense, Activation, Concatenate, Multiply, MaxPool2D, Add,  \
    LSTM, Bidirectional, Conv2D, AveragePooling2D, BatchNormalization, Flatten, GlobalAveragePooling2D, \
    GlobalMaxPooling2D, Reshape, Permute, multiply, Lambda, add, subtract, MaxPooling2D, GRU, ReLU, MaxPooling1D 
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import roc_auc_score, roc_curve
from keras.models import Model, load_model
from keras import backend as K
from sklearn.model_selection import KFold
from keras.layers import Embedding
import matplotlib as mpl
import matplotlib.pyplot as plt
from keras.regularizers import l1, l2
from keras.utils import to_categorical
from keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from Bio import SeqIO
mpl.use('TkAgg')
import warnings
warnings.filterwarnings("ignore")
import math
from tensorflow.keras import layers
from tensorflow import keras
from keras import initializers
from tensorflow.keras.callbacks import Callback

2024-06-12 17:58:38.723376: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-12 17:58:38.955173: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-12 17:58:39.817196: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True  # 不全部占满显存，按需分配当allow_growth设置为True时，分配器将不会指定所有的GPU内存，而是根据需求增长
session = tf.compat.v1.Session(config=config)

2024-06-12 18:05:07.879188: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 8178 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:65:00.0, compute capability: 8.6


In [3]:
def read_fasta(file_path):
    '''File_path: Path to the fasta file
       Returns: List of sequence
    '''
    one=list(SeqIO.parse(file_path,'fasta'))
    return one

In [4]:
def protein_to_Kmer(seqs,K):
    numeric_sequences = []
    base_to_index = {'A': 0, 'C': 1, 'D': 2, 'E': 3,
                     'F': 4, 'G': 5, 'H': 6, 'I': 7, 'K': 8, 'L': 9, 'M': 10,
                     'N': 11, 'P': 12, 'Q': 13, 'R': 14, 'S': 15, 'T': 16, 
                     'V': 17, 'W': 18, 'Y': 19,'X': 20
                     }
    for dna_sequence in seqs:
        numeric_sequence = []
        for base in dna_sequence:
            numeric_sequence.append(base_to_index[base])

        kmer_sequence = []
        for i in range(len(numeric_sequence) - (K - 1)):
            kmer = numeric_sequence[i:i + K]
            kmer_marge = [int("".join(map(str, kmer)))]
            kmer_sequence.append(kmer_marge)

        numeric_sequences.append(kmer_sequence)
    return numeric_sequences

In [5]:
# Read the training set
    
train_pos_seqs = np.array(read_fasta('./data_Kbhb/train_15/Kbhb_pos_15_train.txt'))
train_neg_seqs = np.array(read_fasta('./data_Kbhb/train_15/Kbhb_neg_15_train.txt'))

train_seqs = np.concatenate( (train_pos_seqs, train_neg_seqs), axis=0 )

train = np.array(protein_to_Kmer(train_seqs,1)).astype(np.float32)

train_label = np.array( [1] * 2866 + [0] * 2866 ).astype( np.float32 )
train_label = to_categorical( train_label, num_classes=2 )

# Read the testing set
test_pos_seqs = np.array(read_fasta('./data_Kbhb/train_15/Kbhb_pos_15_test.txt'))
test_neg_seqs = np.array(read_fasta('./data_Kbhb/train_15/Kbhb_neg_15_test.txt'))

test_seqs = np.concatenate((test_pos_seqs, test_neg_seqs), axis=0)

test = np.array(protein_to_Kmer(test_seqs,1)).astype(np.float32)

test_label = np.array([1] * 318 + [0] * 318).astype(np.float32)
test_label = to_categorical(test_label, num_classes=2)

In [6]:
# Performance evaluation
def show_performance(y_true, y_pred):

    TP, FP, FN, TN = 0, 0, 0, 0

    for i in range(len(y_true)):
        if y_true[i] == 1:
            if y_pred[i] > 0.5:
                TP += 1
            else:
                FN += 1
        if y_true[i] == 0:
            if y_pred[i] > 0.5:
                FP += 1
            else:
                TN += 1


    Sn = TP / (TP + FN + 1e-06)

    Sp = TN / (FP + TN + 1e-06)

    Acc = (TP + TN) / len(y_true)

    MCC = ((TP * TN) - (FP * FN)) / np.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN) + 1e-06)

    return Sn, Sp, Acc, MCC

def performance_mean(performance):
    print('Sn = %.4f ± %.4f' % (np.mean(performance[:, 0]), np.std(performance[:, 0])))
    print('Sp = %.4f ± %.4f' % (np.mean(performance[:, 1]), np.std(performance[:, 1])))
    print('Acc = %.4f ± %.4f' % (np.mean(performance[:, 2]), np.std(performance[:, 2])))
    print('Mcc = %.4f ± %.4f' % (np.mean(performance[:, 3]), np.std(performance[:, 3])))
    print('Auc = %.4f ± %.4f' % (np.mean(performance[:, 4]), np.std(performance[:, 4])))

In [7]:
#带有warmup的指数衰减学习率
class WarmupExponentialDecay(Callback):
    def __init__(self,lr_base=0.0002,lr_min=0.0,decay=0,warmup_epochs=0):
        self.num_passed_batchs = 0   #一个计数器
        self.warmup_epochs=warmup_epochs  
        self.lr=lr_base #learning_rate_base
        self.lr_min=lr_min #最小的起始学习率
        self.decay=decay  #指数衰减率
        self.steps_per_epoch=0 #也是一个计数器
    def on_batch_begin(self, batch, logs=None):
        # params是模型自动传递给Callback的一些参数
        if self.steps_per_epoch==0:
            #防止跑验证集的时候呗更改了
            if self.params['steps'] == None:
                self.steps_per_epoch = np.ceil(1. * self.params['samples'] / self.params['batch_size'])
            else:
                self.steps_per_epoch = self.params['steps']
        if self.num_passed_batchs < self.steps_per_epoch * self.warmup_epochs:
            K.set_value(self.model.optimizer.lr,
                        self.lr*(self.num_passed_batchs + 1) / self.steps_per_epoch / self.warmup_epochs)
        else:
            K.set_value(self.model.optimizer.lr,
                        self.lr*((1-self.decay)**(self.num_passed_batchs-self.steps_per_epoch*self.warmup_epochs)))
        self.num_passed_batchs += 1 

In [8]:
# Building the model
def build_model(windows=15,embed_dim=64, weight_decay=1e-4,num_heads=8,ff_dim=64):

    print('*' * 30 + 'Warmup+embedding_64' + '*' * 30)
    input_1 = Input(shape=(windows,))
    # Word embedding coding
    embedding = Embedding(21, embed_dim, input_length=15)
    x_1 = embedding(input_1)
    print(x_1.shape)
    #transformer = TransformerBlock(embed_dim, num_heads, ff_dim)(x_1)
    #print(transformer.shape)

    x_1 = Bidirectional(LSTM(32,return_sequences=True,
                             kernel_regularizer = l2(0.0001), 
                             recurrent_regularizer = l2(0.0001),
                             bias_regularizer = l2(0.0001)))(x_1)
    x_1 = Dropout(0.1)(x_1)
    x_1 = Bidirectional(LSTM(24,return_sequences=True,
                             kernel_regularizer = l2(0.01), 
                             recurrent_regularizer = l2(0.01),
                             bias_regularizer = l2(0.01)))(x_1)
    x_1 = Dropout(0.2)(x_1)
    print(x_1.shape)
    

    #x_1 = SE_Block(x_1,16)
    #x_1 = eca_block(x_1)
     
    #print(x_1.shape)
    
    x_1 = MaxPooling1D(pool_size = 2)(x_1)
    x = Flatten()(x_1)

    x = Dropout(0.2)(x)

    x = Dense(units=2, activation="softmax")(x)


    inputs = [input_1]
    outputs = [x]

    model = Model(inputs=inputs, outputs=outputs, name="Kbhb")

    #optimizer = Adam(learning_rate=1e-4, epsilon=1e-7)
    optimizer = Adam()

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

In [9]:
np.random.seed(0)
tf.random.set_seed(1)  # for reproducibility
# reading model

model = build_model()


BATCH_SIZE = 32
EPOCHS = 200

# # Cross-validation
n = 10
k_fold = KFold(n_splits=n, shuffle=True, random_state=42)

all_performance = []
tprs = []
mean_fpr = np.linspace(0, 1, 100)
for fold_count, (train_index, val_index) in enumerate(k_fold.split(train)):
    print('*' * 30 + ' the ' + str(fold_count + 1) + ' fold ' + '*' * 30)
    trains, val = train[train_index], train[val_index]
    trains_label, val_label = train_label[train_index], train_label[val_index]
    zaoting = EarlyStopping(monitor='val_loss', patience=13, mode='auto')
    xuexilv = WarmupExponentialDecay(lr_base=0.0002,decay=0.00002,warmup_epochs=2)
    callback_lists=[xuexilv,zaoting]
    model.fit(x=trains, y=trains_label, validation_data=(val, val_label), epochs=EPOCHS,
                batch_size=BATCH_SIZE, shuffle=True,
                callbacks=callback_lists,
                verbose=1)
     # 保存模型

    model.save('./warmup_embedding/model_fold' + str(fold_count+1) + '.h5')

    del model

    model = load_model('./warmup_embedding/model_fold' + str(fold_count+1) + '.h5')

    val_pred = model.predict(val, verbose=1)

    # Sn, Sp, Acc, MCC, AUC
    Sn, Sp, Acc, MCC = show_performance(val_label[:, 1], val_pred[:, 1])
    AUC = roc_auc_score(val_label[:, 1], val_pred[:, 1])
    print('Sn = %f, Sp = %f, Acc = %f, MCC = %f, AUC = %f' % (Sn, Sp, Acc, MCC, AUC))

    performance = [Sn, Sp, Acc, MCC, AUC]
    all_performance.append(performance)
    
all_performance = np.array(all_performance)
print('10 fold result:', all_performance)
performance_mean = performance_mean(all_performance)

model.fit(x=train, y=train_label, validation_data=(test, test_label), epochs=EPOCHS,
                      batch_size=BATCH_SIZE, shuffle=True,
                      callbacks=[EarlyStopping(monitor='val_loss', patience=20, mode='auto')],
                      verbose=1)
model.save('./warmup_embedding/model_test.h5')

del model

model = load_model('./warmup_embedding/model_test.h5')

test_score = model.predict(test)


# Sn, Sp, Acc, MCC, AUC
Sn, Sp, Acc, MCC = show_performance(test_label[:,1], test_score[:,1])
AUC = roc_auc_score(test_label[:,1], test_score[:,1])

print('-----------------------------------------------test---------------------------------------')
print('Sn = %f, Sp = %f, Acc = %f, MCC = %f, AUC = %f' % (Sn, Sp, Acc, MCC, AUC))

******************************Warmup+embedding_64******************************
(None, 15, 64)


2024-06-12 18:06:23.816046: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 8178 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:65:00.0, compute capability: 8.6


(None, 15, 48)
****************************** the 1 fold ******************************
Epoch 1/200


2024-06-12 18:06:30.362296: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8800
Could not load symbol cublasGetSmCountTarget from libcublas.so.11. Error: /usr/local/cuda/targets/x86_64-linux/lib/libcublas.so.11: undefined symbol: cublasGetSmCountTarget
2024-06-12 18:06:30.691944: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-06-12 18:06:30.704631: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f21940160f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-06-12 18:06:30.704659: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-06-12 18:06:30.722979: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to e

  1/162 [..............................] - ETA: 18:28 - loss: 3.2008 - accuracy: 0.5312

...

162/162 [==============================] - 4s 26ms/step - loss: 0.7279 - accuracy: 0.7007 - val_loss: 0.7305 - val_accuracy: 0.7056
Epoch 19/200
162/162 [==============================] - 4s 26ms/step - loss: 0.7121 - accuracy: 0.7059 - val_loss: 0.7175 - val_accuracy: 0.7091
Epoch 20/200
162/162 [==============================] - 4s 25ms/step - loss: 0.6951 - accuracy: 0.7100 - val_loss: 0.7011 - val_accuracy: 0.7038
Epoch 21/200
162/162 [==============================] - 4s 25ms/step - loss: 0.6794 - accuracy: 0.7150 - val_loss: 0.6913 - val_accuracy: 0.7108
Epoch 22/200
162/162 [==============================] - 4s 25ms/step - loss: 0.6666 - accuracy: 0.7189 - val_loss: 0.6755 - val_accuracy: 0.7247
Epoch 23/200
162/162 [==============================] - 4s 25ms/step - loss: 0.6527 - accuracy: 0.7243 - val_loss: 0.6748 - val_accuracy: 0.7160
Epoch 24/200
162/162 [==============================] - 4s 25ms/step - loss: 0.6396 - accuracy: 0.7228 - val_loss: 0.6589 - val_accuracy: 0.714

2024-06-12 18:08:52.947401: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.



162/162 [==============================] - 4s 26ms/step - loss: 0.5579 - accuracy: 0.7437 - val_loss: 0.5805 - val_accuracy: 0.7352
Epoch 35/200
162/162 [==============================] - 4s 25ms/step - loss: 0.5528 - accuracy: 0.7480 - val_loss: 0.5683 - val_accuracy: 0.7474
Epoch 36/200
162/162 [==============================] - 4s 25ms/step - loss: 0.5490 - accuracy: 0.7482 - val_loss: 0.5703 - val_accuracy: 0.7404
Epoch 37/200
162/162 [==============================] - 4s 25ms/step - loss: 0.5431 - accuracy: 0.7524 - val_loss: 0.5774 - val_accuracy: 0.7265
Epoch 38/200
162/162 [==============================] - 4s 25ms/step - loss: 0.5367 - accuracy: 0.7536 - val_loss: 0.5631 - val_accuracy: 0.7404
Epoch 39/200
162/162 [==============================] - 4s 25ms/step - loss: 0.5353 - accuracy: 0.7553 - val_loss: 0.5589 - val_accuracy: 0.7491
Epoch 40/200
162/162 [==============================] - 4s 25ms/step - loss: 0.5316 - accuracy: 0.7586 - val_loss: 0.5586 - val_accuracy: 0.740

2024-06-12 18:25:06.163919: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.



180/180 [==============================] - 4s 24ms/step - loss: 0.4369 - accuracy: 0.8133 - val_loss: 0.4618 - val_accuracy: 0.8145
Epoch 3/200
180/180 [==============================] - 4s 25ms/step - loss: 0.4413 - accuracy: 0.8107 - val_loss: 0.4603 - val_accuracy: 0.8145
Epoch 4/200
180/180 [==============================] - 4s 24ms/step - loss: 0.4387 - accuracy: 0.8125 - val_loss: 0.4761 - val_accuracy: 0.8035
Epoch 5/200
180/180 [==============================] - 4s 24ms/step - loss: 0.4369 - accuracy: 0.8144 - val_loss: 0.4589 - val_accuracy: 0.8176
Epoch 6/200
180/180 [==============================] - 4s 24ms/step - loss: 0.4385 - accuracy: 0.8098 - val_loss: 0.4640 - val_accuracy: 0.8113
Epoch 7/200
180/180 [==============================] - 4s 24ms/step - loss: 0.4330 - accuracy: 0.8139 - val_loss: 0.4647 - val_accuracy: 0.8050
Epoch 8/200
180/180 [==============================] - 4s 24ms/step - loss: 0.4377 - accuracy: 0.8137 - val_loss: 0.4742 - val_accuracy: 0.8003
Epoc

In [10]:
df = pd.DataFrame({
    'true_label': test_label[:,1],
    'predicted_probability': test_score[:, 1]  # 提取正类的预测概率
})
df.to_csv('./warmup_embedding/predictions_with_probabilities_bilstm.csv', index=False)

In [9]:
np.random.seed(0)
tf.random.set_seed(1)  # for reproducibility
# reading model

model = build_model()


BATCH_SIZE = 32
EPOCHS = 200

# # Cross-validation
n = 10
k_fold = KFold(n_splits=n, shuffle=True, random_state=42)

all_performance = []
tprs = []
mean_fpr = np.linspace(0, 1, 100)
for fold_count, (train_index, val_index) in enumerate(k_fold.split(train)):
    print('*' * 30 + ' the ' + str(fold_count + 1) + ' fold ' + '*' * 30)
    trains, val = train[train_index], train[val_index]
    trains_label, val_label = train_label[train_index], train_label[val_index]
    zaoting = EarlyStopping(monitor='val_loss', patience=13, mode='auto')
    xuexilv = WarmupExponentialDecay(lr_base=0.0002,decay=0.00002,warmup_epochs=2)
    callback_lists=[xuexilv,zaoting]
    model.fit(x=trains, y=trains_label, validation_data=(val, val_label), epochs=EPOCHS,
                batch_size=BATCH_SIZE, shuffle=True,
                callbacks=callback_lists,
                verbose=1)
     # 保存模型

    model.save('./warmup_embedding/model_fold' + str(fold_count+1) + '.h5')

    del model

    model = load_model('./warmup_embedding/model_fold' + str(fold_count+1) + '.h5')

    val_pred = model.predict(val, verbose=1)

    # Sn, Sp, Acc, MCC, AUC
    Sn, Sp, Acc, MCC = show_performance(val_label[:, 1], val_pred[:, 1])
    AUC = roc_auc_score(val_label[:, 1], val_pred[:, 1])
    print('Sn = %f, Sp = %f, Acc = %f, MCC = %f, AUC = %f' % (Sn, Sp, Acc, MCC, AUC))

    performance = [Sn, Sp, Acc, MCC, AUC]
    all_performance.append(performance)
    
all_performance = np.array(all_performance)
print('10 fold result:', all_performance)
performance_mean = performance_mean(all_performance)

model.fit(x=train, y=train_label, validation_data=(test, test_label), epochs=EPOCHS,
                      batch_size=BATCH_SIZE, shuffle=True,
                      callbacks=[EarlyStopping(monitor='val_loss', patience=20, mode='auto')],
                      verbose=1)
model.save('./warmup_embedding/model_test.h5')

del model

model = load_model('./warmup_embedding/model_test.h5')

test_score = model.predict(test)


# Sn, Sp, Acc, MCC, AUC
Sn, Sp, Acc, MCC = show_performance(test_label[:,1], test_score[:,1])
AUC = roc_auc_score(test_label[:,1], test_score[:,1])

print('-----------------------------------------------test---------------------------------------')
print('Sn = %f, Sp = %f, Acc = %f, MCC = %f, AUC = %f' % (Sn, Sp, Acc, MCC, AUC))

******************************Warmup+embedding_64******************************
(None, 15, 64)


2024-06-06 09:53:30.674327: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3954 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:65:00.0, compute capability: 8.6


(None, 15, 48)
****************************** the 1 fold ******************************
Epoch 1/200


2024-06-06 09:53:38.745424: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8800
Could not load symbol cublasGetSmCountTarget from libcublas.so.11. Error: /usr/local/cuda/targets/x86_64-linux/lib/libcublas.so.11: undefined symbol: cublasGetSmCountTarget
2024-06-06 09:53:39.016807: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-06-06 09:53:39.056488: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fe9340fc320 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-06-06 09:53:39.056528: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-06-06 09:53:39.061533: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to e

  1/162 [..............................] - ETA: 21:43 - loss: 3.2261 - accuracy: 0.4062

...

162/162 [==============================] - 9s 54ms/step - loss: 0.7368 - accuracy: 0.6931 - val_loss: 0.7423 - val_accuracy: 0.6934
Epoch 19/200
162/162 [==============================] - 8s 52ms/step - loss: 0.7225 - accuracy: 0.6950 - val_loss: 0.7245 - val_accuracy: 0.6916
Epoch 20/200
162/162 [==============================] - 9s 53ms/step - loss: 0.7078 - accuracy: 0.6995 - val_loss: 0.7122 - val_accuracy: 0.6934
Epoch 21/200
162/162 [==============================] - 9s 53ms/step - loss: 0.6945 - accuracy: 0.6968 - val_loss: 0.7006 - val_accuracy: 0.6934
Epoch 22/200
162/162 [==============================] - 8s 52ms/step - loss: 0.6828 - accuracy: 0.7010 - val_loss: 0.6858 - val_accuracy: 0.7160
Epoch 23/200
162/162 [==============================] - 9s 53ms/step - loss: 0.6709 - accuracy: 0.7009 - val_loss: 0.6800 - val_accuracy: 0.6951
Epoch 24/200
162/162 [==============================] - 8s 51ms/step - loss: 0.6587 - accuracy: 0.7067 - val_loss: 0.6678 - val_accuracy: 0.705

2024-06-06 09:58:30.045486: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.



162/162 [==============================] - 9s 53ms/step - loss: 0.5683 - accuracy: 0.7394 - val_loss: 0.5859 - val_accuracy: 0.7369
Epoch 35/200
162/162 [==============================] - 8s 52ms/step - loss: 0.5624 - accuracy: 0.7410 - val_loss: 0.5750 - val_accuracy: 0.7474
Epoch 36/200
162/162 [==============================] - 9s 53ms/step - loss: 0.5590 - accuracy: 0.7379 - val_loss: 0.5751 - val_accuracy: 0.7544
Epoch 37/200
162/162 [==============================] - 8s 52ms/step - loss: 0.5533 - accuracy: 0.7534 - val_loss: 0.5842 - val_accuracy: 0.7387
Epoch 38/200
162/162 [==============================] - 8s 52ms/step - loss: 0.5484 - accuracy: 0.7530 - val_loss: 0.5688 - val_accuracy: 0.7526
Epoch 39/200
162/162 [==============================] - 8s 52ms/step - loss: 0.5452 - accuracy: 0.7516 - val_loss: 0.5664 - val_accuracy: 0.7561
Epoch 40/200
162/162 [==============================] - 9s 53ms/step - loss: 0.5391 - accuracy: 0.7557 - val_loss: 0.5622 - val_accuracy: 0.754

2024-06-06 10:22:54.372437: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.



180/180 [==============================] - 8s 45ms/step - loss: 0.4464 - accuracy: 0.8104 - val_loss: 0.4720 - val_accuracy: 0.8066
Epoch 3/200
180/180 [==============================] - 8s 46ms/step - loss: 0.4506 - accuracy: 0.8057 - val_loss: 0.4733 - val_accuracy: 0.8003
Epoch 4/200
180/180 [==============================] - 9s 48ms/step - loss: 0.4460 - accuracy: 0.8069 - val_loss: 0.4768 - val_accuracy: 0.8050
Epoch 5/200
180/180 [==============================] - 8s 47ms/step - loss: 0.4466 - accuracy: 0.8102 - val_loss: 0.4734 - val_accuracy: 0.7956
Epoch 6/200
180/180 [==============================] - 8s 46ms/step - loss: 0.4450 - accuracy: 0.8077 - val_loss: 0.4759 - val_accuracy: 0.7940
Epoch 7/200
180/180 [==============================] - 8s 46ms/step - loss: 0.4465 - accuracy: 0.8098 - val_loss: 0.4698 - val_accuracy: 0.8050
Epoch 8/200
180/180 [==============================] - 8s 47ms/step - loss: 0.4458 - accuracy: 0.8086 - val_loss: 0.4744 - val_accuracy: 0.7956
Epoc

In [9]:
np.random.seed(0)
tf.random.set_seed(1)  # for reproducibility
# reading model

model = build_model()


BATCH_SIZE = 32
EPOCHS = 200

# # Cross-validation
n = 10
k_fold = KFold(n_splits=n, shuffle=True, random_state=42)

all_performance = []
tprs = []
mean_fpr = np.linspace(0, 1, 100)
for fold_count, (train_index, val_index) in enumerate(k_fold.split(train)):
    print('*' * 30 + ' the ' + str(fold_count + 1) + ' fold ' + '*' * 30)
    trains, val = train[train_index], train[val_index]
    trains_label, val_label = train_label[train_index], train_label[val_index]
    zaoting = EarlyStopping(monitor='val_loss', patience=13, mode='auto')
    xuexilv = WarmupExponentialDecay(lr_base=0.0002,decay=0.00002,warmup_epochs=2)
    callback_lists=[xuexilv,zaoting]
    model.fit(x=trains, y=trains_label, validation_data=(val, val_label), epochs=EPOCHS,
                batch_size=BATCH_SIZE, shuffle=True,
                callbacks=callback_lists,
                verbose=1)
     # 保存模型

    model.save('./warmup_embedding/model_fold' + str(fold_count+1) + '.h5')

    del model

    model = load_model('./warmup_embedding/model_fold' + str(fold_count+1) + '.h5')

    val_pred = model.predict(val, verbose=1)

    # Sn, Sp, Acc, MCC, AUC
    Sn, Sp, Acc, MCC = show_performance(val_label[:, 1], val_pred[:, 1])
    AUC = roc_auc_score(val_label[:, 1], val_pred[:, 1])
    print('Sn = %f, Sp = %f, Acc = %f, MCC = %f, AUC = %f' % (Sn, Sp, Acc, MCC, AUC))

    performance = [Sn, Sp, Acc, MCC, AUC]
    all_performance.append(performance)
    
all_performance = np.array(all_performance)
print('10 fold result:', all_performance)
performance_mean = performance_mean(all_performance)

model.fit(x=train, y=train_label, validation_data=(test, test_label), epochs=EPOCHS,
                      batch_size=BATCH_SIZE, shuffle=True,
                      callbacks=[EarlyStopping(monitor='val_loss', patience=20, mode='auto')],
                      verbose=1)
model.save('./warmup_embedding/model_test.h5')

del model

model = load_model('./warmup_embedding/model_test.h5')

test_score = model.predict(test)


# Sn, Sp, Acc, MCC, AUC
Sn, Sp, Acc, MCC = show_performance(test_label[:,1], test_score[:,1])
AUC = roc_auc_score(test_label[:,1], test_score[:,1])

print('-----------------------------------------------test---------------------------------------')
print('Sn = %f, Sp = %f, Acc = %f, MCC = %f, AUC = %f' % (Sn, Sp, Acc, MCC, AUC))

******************************Warmup+embedding_64******************************
(None, 15, 64)


2024-06-06 08:42:14.780925: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5110 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:65:00.0, compute capability: 8.6


(None, 15, 48)
****************************** the 1 fold ******************************
Epoch 1/200


2024-06-06 08:42:21.853236: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8800
Could not load symbol cublasGetSmCountTarget from libcublas.so.11. Error: /usr/local/cuda/targets/x86_64-linux/lib/libcublas.so.11: undefined symbol: cublasGetSmCountTarget
2024-06-06 08:42:22.065724: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-06-06 08:42:22.092370: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f5fec04fe80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-06-06 08:42:22.092411: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-06-06 08:42:22.100465: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to e

  1/162 [..............................] - ETA: 18:49 - loss: 3.2254 - accuracy: 0.3750

...

162/162 [==============================] - 6s 37ms/step - loss: 0.7316 - accuracy: 0.6950 - val_loss: 0.7339 - val_accuracy: 0.7056
Epoch 19/200
162/162 [==============================] - 6s 36ms/step - loss: 0.7174 - accuracy: 0.6968 - val_loss: 0.7190 - val_accuracy: 0.7003
Epoch 20/200
162/162 [==============================] - 6s 36ms/step - loss: 0.7031 - accuracy: 0.7001 - val_loss: 0.7054 - val_accuracy: 0.7056
Epoch 21/200
162/162 [==============================] - 6s 36ms/step - loss: 0.6887 - accuracy: 0.7082 - val_loss: 0.6933 - val_accuracy: 0.7108
Epoch 22/200
162/162 [==============================] - 6s 36ms/step - loss: 0.6780 - accuracy: 0.7043 - val_loss: 0.6831 - val_accuracy: 0.7143
Epoch 23/200
162/162 [==============================] - 6s 36ms/step - loss: 0.6651 - accuracy: 0.7073 - val_loss: 0.6779 - val_accuracy: 0.7038
Epoch 24/200
162/162 [==============================] - 6s 36ms/step - loss: 0.6528 - accuracy: 0.7076 - val_loss: 0.6662 - val_accuracy: 0.705

2024-06-06 08:45:41.633016: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.



162/162 [==============================] - 6s 37ms/step - loss: 0.5701 - accuracy: 0.7346 - val_loss: 0.5946 - val_accuracy: 0.7300
Epoch 35/200
162/162 [==============================] - 6s 36ms/step - loss: 0.5638 - accuracy: 0.7451 - val_loss: 0.5819 - val_accuracy: 0.7404
Epoch 36/200
162/162 [==============================] - 6s 36ms/step - loss: 0.5615 - accuracy: 0.7334 - val_loss: 0.5833 - val_accuracy: 0.7334
Epoch 37/200
162/162 [==============================] - 6s 36ms/step - loss: 0.5549 - accuracy: 0.7445 - val_loss: 0.5946 - val_accuracy: 0.7160
Epoch 38/200
162/162 [==============================] - 6s 36ms/step - loss: 0.5506 - accuracy: 0.7462 - val_loss: 0.5770 - val_accuracy: 0.7282
Epoch 39/200
162/162 [==============================] - 6s 36ms/step - loss: 0.5479 - accuracy: 0.7460 - val_loss: 0.5740 - val_accuracy: 0.7422
Epoch 40/200
162/162 [==============================] - 6s 36ms/step - loss: 0.5429 - accuracy: 0.7487 - val_loss: 0.5683 - val_accuracy: 0.745

2024-06-06 09:10:28.492074: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.



180/180 [==============================] - 6s 32ms/step - loss: 0.4371 - accuracy: 0.8137 - val_loss: 0.4623 - val_accuracy: 0.8082
Epoch 3/200
180/180 [==============================] - 6s 33ms/step - loss: 0.4382 - accuracy: 0.8145 - val_loss: 0.4578 - val_accuracy: 0.8145
Epoch 4/200
180/180 [==============================] - 6s 32ms/step - loss: 0.4366 - accuracy: 0.8123 - val_loss: 0.4724 - val_accuracy: 0.8176
Epoch 5/200
180/180 [==============================] - 6s 32ms/step - loss: 0.4359 - accuracy: 0.8135 - val_loss: 0.4639 - val_accuracy: 0.8097
Epoch 6/200
180/180 [==============================] - 6s 32ms/step - loss: 0.4343 - accuracy: 0.8224 - val_loss: 0.4609 - val_accuracy: 0.8097
Epoch 7/200
180/180 [==============================] - 6s 32ms/step - loss: 0.4345 - accuracy: 0.8173 - val_loss: 0.4606 - val_accuracy: 0.8082
Epoch 8/200
180/180 [==============================] - 6s 32ms/step - loss: 0.4360 - accuracy: 0.8109 - val_loss: 0.4738 - val_accuracy: 0.8082
Epoc

In [9]:
np.random.seed(0)
tf.random.set_seed(1)  # for reproducibility
# reading model

model = build_model()


BATCH_SIZE = 32
EPOCHS = 200

# # Cross-validation
n = 10
k_fold = KFold(n_splits=n, shuffle=True, random_state=42)

all_performance = []
tprs = []
mean_fpr = np.linspace(0, 1, 100)
for fold_count, (train_index, val_index) in enumerate(k_fold.split(train)):
    print('*' * 30 + ' the ' + str(fold_count + 1) + ' fold ' + '*' * 30)
    trains, val = train[train_index], train[val_index]
    trains_label, val_label = train_label[train_index], train_label[val_index]
    zaoting = EarlyStopping(monitor='val_loss', patience=13, mode='auto')
    xuexilv = WarmupExponentialDecay(lr_base=0.0002,decay=0.00002,warmup_epochs=2)
    callback_lists=[xuexilv,zaoting]
    model.fit(x=trains, y=trains_label, validation_data=(val, val_label), epochs=EPOCHS,
                batch_size=BATCH_SIZE, shuffle=True,
                callbacks=callback_lists,
                verbose=1)
     # 保存模型

    model.save('./warmup_embedding/model_fold' + str(fold_count+1) + '.h5')

    del model

    model = load_model('./warmup_embedding/model_fold' + str(fold_count+1) + '.h5')

    val_pred = model.predict(val, verbose=1)

    # Sn, Sp, Acc, MCC, AUC
    Sn, Sp, Acc, MCC = show_performance(val_label[:, 1], val_pred[:, 1])
    AUC = roc_auc_score(val_label[:, 1], val_pred[:, 1])
    print('Sn = %f, Sp = %f, Acc = %f, MCC = %f, AUC = %f' % (Sn, Sp, Acc, MCC, AUC))

    performance = [Sn, Sp, Acc, MCC, AUC]
    all_performance.append(performance)
    
all_performance = np.array(all_performance)
print('10 fold result:', all_performance)
performance_mean = performance_mean(all_performance)

model.fit(x=train, y=train_label, validation_data=(test, test_label), epochs=EPOCHS,
                      batch_size=BATCH_SIZE, shuffle=True,
                      callbacks=[EarlyStopping(monitor='val_loss', patience=20, mode='auto')],
                      verbose=1)
model.save('./warmup_embedding/model_test.h5')

del model

model = load_model('./warmup_embedding/model_test.h5')

test_score = model.predict(test)


# Sn, Sp, Acc, MCC, AUC
Sn, Sp, Acc, MCC = show_performance(test_label[:,1], test_score[:,1])
AUC = roc_auc_score(test_label[:,1], test_score[:,1])

print('-----------------------------------------------test---------------------------------------')
print('Sn = %f, Sp = %f, Acc = %f, MCC = %f, AUC = %f' % (Sn, Sp, Acc, MCC, AUC))

******************************Warmup+embedding_256******************************
(None, 15, 256)


2024-06-05 19:13:22.348384: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4226 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:65:00.0, compute capability: 8.6


(None, 15, 48)
****************************** the 1 fold ******************************
Epoch 1/200


2024-06-05 19:13:30.496253: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8800
Could not load symbol cublasGetSmCountTarget from libcublas.so.11. Error: /usr/local/cuda/targets/x86_64-linux/lib/libcublas.so.11: undefined symbol: cublasGetSmCountTarget
2024-06-05 19:13:30.761179: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-06-05 19:13:30.799547: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f0dd800df80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-06-05 19:13:30.799577: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-06-05 19:13:30.805921: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to e

  1/162 [..............................] - ETA: 22:09 - loss: 3.2566 - accuracy: 0.7188

...

162/162 [==============================] - 9s 55ms/step - loss: 0.7001 - accuracy: 0.7336 - val_loss: 0.7200 - val_accuracy: 0.7265
Epoch 19/200
162/162 [==============================] - 9s 53ms/step - loss: 0.6823 - accuracy: 0.7358 - val_loss: 0.7082 - val_accuracy: 0.7195
Epoch 20/200
162/162 [==============================] - 8s 52ms/step - loss: 0.6656 - accuracy: 0.7431 - val_loss: 0.6900 - val_accuracy: 0.7334
Epoch 21/200
162/162 [==============================] - 9s 53ms/step - loss: 0.6510 - accuracy: 0.7451 - val_loss: 0.6809 - val_accuracy: 0.7369
Epoch 22/200
162/162 [==============================] - 9s 54ms/step - loss: 0.6371 - accuracy: 0.7497 - val_loss: 0.6628 - val_accuracy: 0.7404
Epoch 23/200
162/162 [==============================] - 8s 52ms/step - loss: 0.6253 - accuracy: 0.7499 - val_loss: 0.6669 - val_accuracy: 0.7230
Epoch 24/200
162/162 [==============================] - 8s 52ms/step - loss: 0.6119 - accuracy: 0.7511 - val_loss: 0.6519 - val_accuracy: 0.724

2024-06-05 19:18:26.477365: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.



162/162 [==============================] - 9s 54ms/step - loss: 0.5310 - accuracy: 0.7761 - val_loss: 0.5718 - val_accuracy: 0.7491
Epoch 35/200
162/162 [==============================] - 9s 53ms/step - loss: 0.5268 - accuracy: 0.7726 - val_loss: 0.5660 - val_accuracy: 0.7544
Epoch 36/200
162/162 [==============================] - 9s 53ms/step - loss: 0.5242 - accuracy: 0.7759 - val_loss: 0.5663 - val_accuracy: 0.7561
Epoch 37/200
162/162 [==============================] - 9s 54ms/step - loss: 0.5156 - accuracy: 0.7761 - val_loss: 0.5710 - val_accuracy: 0.7526
Epoch 38/200
162/162 [==============================] - 9s 53ms/step - loss: 0.5111 - accuracy: 0.7854 - val_loss: 0.5583 - val_accuracy: 0.7596
Epoch 39/200
162/162 [==============================] - 9s 53ms/step - loss: 0.5096 - accuracy: 0.7763 - val_loss: 0.5564 - val_accuracy: 0.7596
Epoch 40/200
162/162 [==============================] - 9s 53ms/step - loss: 0.5054 - accuracy: 0.7796 - val_loss: 0.5522 - val_accuracy: 0.759

2024-06-05 19:44:19.602488: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.



180/180 [==============================] - 9s 49ms/step - loss: 0.3324 - accuracy: 0.8843 - val_loss: 0.6547 - val_accuracy: 0.7390
Epoch 3/200
180/180 [==============================] - 9s 48ms/step - loss: 0.3263 - accuracy: 0.8904 - val_loss: 0.6525 - val_accuracy: 0.7343
Epoch 4/200
180/180 [==============================] - 8s 47ms/step - loss: 0.3264 - accuracy: 0.8920 - val_loss: 0.6421 - val_accuracy: 0.7500
Epoch 5/200
180/180 [==============================] - 8s 47ms/step - loss: 0.3195 - accuracy: 0.8892 - val_loss: 0.6428 - val_accuracy: 0.7657
Epoch 6/200
180/180 [==============================] - 8s 47ms/step - loss: 0.3147 - accuracy: 0.8974 - val_loss: 0.6625 - val_accuracy: 0.7563
Epoch 7/200
180/180 [==============================] - 8s 47ms/step - loss: 0.3191 - accuracy: 0.8945 - val_loss: 0.6559 - val_accuracy: 0.7547
Epoch 8/200
180/180 [==============================] - 8s 46ms/step - loss: 0.3154 - accuracy: 0.8938 - val_loss: 0.6704 - val_accuracy: 0.7327
Epoc

In [12]:
np.random.seed(0)
tf.random.set_seed(1)  # for reproducibility
# reading model

model = build_model()


BATCH_SIZE = 32
EPOCHS = 200

# # Cross-validation
n = 10
k_fold = KFold(n_splits=n, shuffle=True, random_state=42)

all_performance = []
tprs = []
mean_fpr = np.linspace(0, 1, 100)
for fold_count, (train_index, val_index) in enumerate(k_fold.split(train)):
    print('*' * 30 + ' the ' + str(fold_count + 1) + ' fold ' + '*' * 30)
    trains, val = train[train_index], train[val_index]
    trains_label, val_label = train_label[train_index], train_label[val_index]
    zaoting = EarlyStopping(monitor='val_loss', patience=13, mode='auto')
    xuexilv = WarmupExponentialDecay(lr_base=0.0002,decay=0.00002,warmup_epochs=2)
    callback_lists=[xuexilv,zaoting]
    model.fit(x=trains, y=trains_label, validation_data=(val, val_label), epochs=EPOCHS,
                batch_size=BATCH_SIZE, shuffle=True,
                callbacks=callback_lists,
                verbose=1)
     # 保存模型

    model.save('./warmup_embedding/model_fold' + str(fold_count+1) + '.h5')

    del model

    model = load_model('./warmup_embedding/model_fold' + str(fold_count+1) + '.h5')

    val_pred = model.predict(val, verbose=1)

    # Sn, Sp, Acc, MCC, AUC
    Sn, Sp, Acc, MCC = show_performance(val_label[:, 1], val_pred[:, 1])
    AUC = roc_auc_score(val_label[:, 1], val_pred[:, 1])
    print('Sn = %f, Sp = %f, Acc = %f, MCC = %f, AUC = %f' % (Sn, Sp, Acc, MCC, AUC))

    performance = [Sn, Sp, Acc, MCC, AUC]
    all_performance.append(performance)
    
all_performance = np.array(all_performance)
print('10 fold result:', all_performance)
performance_mean = performance_mean(all_performance)

model.fit(x=train, y=train_label, validation_data=(test, test_label), epochs=EPOCHS,
                      batch_size=BATCH_SIZE, shuffle=True,
                      callbacks=[EarlyStopping(monitor='val_loss', patience=20, mode='auto')],
                      verbose=1)
model.save('./warmup_embedding/model_test.h5')

del model

model = load_model('./warmup_embedding/model_test.h5')

test_score = model.predict(test)


# Sn, Sp, Acc, MCC, AUC
Sn, Sp, Acc, MCC = show_performance(test_label[:,1], test_score[:,1])
AUC = roc_auc_score(test_label[:,1], test_score[:,1])

print('-----------------------------------------------test---------------------------------------')
print('Sn = %f, Sp = %f, Acc = %f, MCC = %f, AUC = %f' % (Sn, Sp, Acc, MCC, AUC))

******************************Warmup+embedding_128******************************
(None, 15, 128)
(None, 15, 48)
****************************** the 1 fold ******************************
Epoch 1/200


2024-06-05 18:31:40.905151: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8800
Could not load symbol cublasGetSmCountTarget from libcublas.so.11. Error: /usr/local/cuda/targets/x86_64-linux/lib/libcublas.so.11: undefined symbol: cublasGetSmCountTarget
2024-06-05 18:31:41.187895: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-06-05 18:31:41.226247: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x560efccbbef0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-06-05 18:31:41.226270: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-06-05 18:31:41.230228: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to e

  1/162 [..............................] - ETA: 22:21 - loss: 3.2359 - accuracy: 0.5000

2024-06-05 18:31:41.878539: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

2024-06-05 18:31:41.899499: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

2024-06-05 18:31:41.901901: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

2024-06-05 18:31:41.942265: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before

  4/162 [..............................] - ETA: 14s - loss: 3.2350 - accuracy: 0.5703 

...

162/162 [==============================] - 9s 53ms/step - loss: 0.7004 - accuracy: 0.7317 - val_loss: 0.7189 - val_accuracy: 0.7230
Epoch 19/200
162/162 [==============================] - 9s 53ms/step - loss: 0.6840 - accuracy: 0.7319 - val_loss: 0.7067 - val_accuracy: 0.7125
Epoch 20/200
162/162 [==============================] - 9s 53ms/step - loss: 0.6691 - accuracy: 0.7373 - val_loss: 0.6877 - val_accuracy: 0.7230
Epoch 21/200
162/162 [==============================] - 9s 53ms/step - loss: 0.6537 - accuracy: 0.7420 - val_loss: 0.6782 - val_accuracy: 0.7230
Epoch 22/200
162/162 [==============================] - 8s 52ms/step - loss: 0.6431 - accuracy: 0.7439 - val_loss: 0.6595 - val_accuracy: 0.7265
Epoch 23/200
162/162 [==============================] - 8s 52ms/step - loss: 0.6307 - accuracy: 0.7441 - val_loss: 0.6604 - val_accuracy: 0.7160
Epoch 24/200
162/162 [==============================] - 9s 54ms/step - loss: 0.6196 - accuracy: 0.7458 - val_loss: 0.6452 - val_accuracy: 0.740

2024-06-05 18:36:36.164591: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.



162/162 [==============================] - 9s 54ms/step - loss: 0.5414 - accuracy: 0.7619 - val_loss: 0.5687 - val_accuracy: 0.7544
Epoch 35/200
162/162 [==============================] - 9s 53ms/step - loss: 0.5370 - accuracy: 0.7625 - val_loss: 0.5566 - val_accuracy: 0.7613
Epoch 36/200
162/162 [==============================] - 9s 53ms/step - loss: 0.5325 - accuracy: 0.7631 - val_loss: 0.5617 - val_accuracy: 0.7491
Epoch 37/200
162/162 [==============================] - 9s 53ms/step - loss: 0.5278 - accuracy: 0.7658 - val_loss: 0.5696 - val_accuracy: 0.7317
Epoch 38/200
162/162 [==============================] - 9s 53ms/step - loss: 0.5240 - accuracy: 0.7637 - val_loss: 0.5565 - val_accuracy: 0.7544
Epoch 39/200
162/162 [==============================] - 9s 53ms/step - loss: 0.5235 - accuracy: 0.7652 - val_loss: 0.5497 - val_accuracy: 0.7578
Epoch 40/200
162/162 [==============================] - 9s 53ms/step - loss: 0.5187 - accuracy: 0.7660 - val_loss: 0.5491 - val_accuracy: 0.756

2024-06-05 19:06:42.411074: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.



180/180 [==============================] - 9s 48ms/step - loss: 0.4130 - accuracy: 0.8343 - val_loss: 0.4971 - val_accuracy: 0.7925
Epoch 3/200
180/180 [==============================] - 9s 49ms/step - loss: 0.4158 - accuracy: 0.8304 - val_loss: 0.4854 - val_accuracy: 0.7956
Epoch 4/200
180/180 [==============================] - 9s 48ms/step - loss: 0.4135 - accuracy: 0.8334 - val_loss: 0.5059 - val_accuracy: 0.7862
Epoch 5/200
180/180 [==============================] - 9s 47ms/step - loss: 0.4105 - accuracy: 0.8341 - val_loss: 0.4934 - val_accuracy: 0.7893
Epoch 6/200
180/180 [==============================] - 9s 48ms/step - loss: 0.4132 - accuracy: 0.8346 - val_loss: 0.4908 - val_accuracy: 0.8003
Epoch 7/200
180/180 [==============================] - 8s 47ms/step - loss: 0.4108 - accuracy: 0.8355 - val_loss: 0.4881 - val_accuracy: 0.7956
Epoch 8/200
180/180 [==============================] - 8s 46ms/step - loss: 0.4125 - accuracy: 0.8332 - val_loss: 0.5043 - val_accuracy: 0.7877
Epoc

In [9]:
np.random.seed(0)
tf.random.set_seed(1)  # for reproducibility
# reading model

model = build_model()


BATCH_SIZE = 32
EPOCHS = 200

# # Cross-validation
n = 10
k_fold = KFold(n_splits=n, shuffle=True, random_state=42)

all_performance = []
tprs = []
mean_fpr = np.linspace(0, 1, 100)
for fold_count, (train_index, val_index) in enumerate(k_fold.split(train)):
    print('*' * 30 + ' the ' + str(fold_count + 1) + ' fold ' + '*' * 30)
    trains, val = train[train_index], train[val_index]
    trains_label, val_label = train_label[train_index], train_label[val_index]
    zaoting = EarlyStopping(monitor='val_loss', patience=13, mode='auto')
    xuexilv = WarmupExponentialDecay(lr_base=0.0002,decay=0.00002,warmup_epochs=2)
    callback_lists=[xuexilv,zaoting]
    model.fit(x=trains, y=trains_label, validation_data=(val, val_label), epochs=EPOCHS,
                batch_size=BATCH_SIZE, shuffle=True,
                callbacks=callback_lists,
                verbose=1)
     # 保存模型

    model.save('./warmup_embedding/model_fold' + str(fold_count+1) + '.h5')

    del model

    model = load_model('./warmup_embedding/model_fold' + str(fold_count+1) + '.h5')

    val_pred = model.predict(val, verbose=1)

    # Sn, Sp, Acc, MCC, AUC
    Sn, Sp, Acc, MCC = show_performance(val_label[:, 1], val_pred[:, 1])
    AUC = roc_auc_score(val_label[:, 1], val_pred[:, 1])
    print('Sn = %f, Sp = %f, Acc = %f, MCC = %f, AUC = %f' % (Sn, Sp, Acc, MCC, AUC))

    performance = [Sn, Sp, Acc, MCC, AUC]
    all_performance.append(performance)
    
all_performance = np.array(all_performance)
print('10 fold result:', all_performance)
performance_mean = performance_mean(all_performance)

model.fit(x=train, y=train_label, validation_data=(test, test_label), epochs=EPOCHS,
                      batch_size=BATCH_SIZE, shuffle=True,
                      callbacks=[EarlyStopping(monitor='val_loss', patience=20, mode='auto')],
                      verbose=1)
model.save('./warmup_embedding/model_test.h5')

del model

model = load_model('./warmup_embedding/model_test.h5')

test_score = model.predict(test)


# Sn, Sp, Acc, MCC, AUC
Sn, Sp, Acc, MCC = show_performance(test_label[:,1], test_score[:,1])
AUC = roc_auc_score(test_label[:,1], test_score[:,1])

print('-----------------------------------------------test---------------------------------------')
print('Sn = %f, Sp = %f, Acc = %f, MCC = %f, AUC = %f' % (Sn, Sp, Acc, MCC, AUC))

******************************Warmup+embedding_64******************************
(None, 15, 64)


2024-06-05 16:01:18.060290: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4227 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:65:00.0, compute capability: 8.6


(None, 15, 48)
****************************** the 1 fold ******************************
Epoch 1/200


2024-06-05 16:01:26.173254: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8800
Could not load symbol cublasGetSmCountTarget from libcublas.so.11. Error: /usr/local/cuda/targets/x86_64-linux/lib/libcublas.so.11: undefined symbol: cublasGetSmCountTarget
2024-06-05 16:01:26.466588: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-06-05 16:01:26.506313: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f39b427efa0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-06-05 16:01:26.506337: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-06-05 16:01:26.509974: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to e

  1/162 [..............................] - ETA: 22:01 - loss: 3.2462 - accuracy: 0.4375

...

162/162 [==============================] - 9s 54ms/step - loss: 0.7270 - accuracy: 0.6997 - val_loss: 0.7333 - val_accuracy: 0.6969
Epoch 19/200
162/162 [==============================] - 9s 53ms/step - loss: 0.7108 - accuracy: 0.7045 - val_loss: 0.7173 - val_accuracy: 0.7073
Epoch 20/200
162/162 [==============================] - 9s 53ms/step - loss: 0.6942 - accuracy: 0.7125 - val_loss: 0.7014 - val_accuracy: 0.7056
Epoch 21/200
162/162 [==============================] - 9s 54ms/step - loss: 0.6780 - accuracy: 0.7142 - val_loss: 0.6902 - val_accuracy: 0.7160
Epoch 22/200
162/162 [==============================] - 9s 53ms/step - loss: 0.6664 - accuracy: 0.7179 - val_loss: 0.6786 - val_accuracy: 0.7195
Epoch 23/200
162/162 [==============================] - 9s 53ms/step - loss: 0.6525 - accuracy: 0.7202 - val_loss: 0.6764 - val_accuracy: 0.7091
Epoch 24/200
162/162 [==============================] - 9s 53ms/step - loss: 0.6398 - accuracy: 0.7272 - val_loss: 0.6647 - val_accuracy: 0.709

2024-06-05 16:06:23.755051: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.



162/162 [==============================] - 9s 54ms/step - loss: 0.5559 - accuracy: 0.7460 - val_loss: 0.5836 - val_accuracy: 0.7334
Epoch 35/200
162/162 [==============================] - 9s 53ms/step - loss: 0.5496 - accuracy: 0.7536 - val_loss: 0.5697 - val_accuracy: 0.7439
Epoch 36/200
162/162 [==============================] - 9s 53ms/step - loss: 0.5456 - accuracy: 0.7518 - val_loss: 0.5727 - val_accuracy: 0.7369
Epoch 37/200
162/162 [==============================] - 9s 53ms/step - loss: 0.5404 - accuracy: 0.7561 - val_loss: 0.5825 - val_accuracy: 0.7230
Epoch 38/200
162/162 [==============================] - 9s 53ms/step - loss: 0.5378 - accuracy: 0.7544 - val_loss: 0.5647 - val_accuracy: 0.7334
Epoch 39/200
162/162 [==============================] - 9s 54ms/step - loss: 0.5361 - accuracy: 0.7579 - val_loss: 0.5596 - val_accuracy: 0.7474
Epoch 40/200
162/162 [==============================] - 9s 54ms/step - loss: 0.5296 - accuracy: 0.7631 - val_loss: 0.5571 - val_accuracy: 0.749

2024-06-05 16:42:50.614596: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.



180/180 [==============================] - 8s 47ms/step - loss: 0.4307 - accuracy: 0.8210 - val_loss: 0.4800 - val_accuracy: 0.7940
Epoch 3/200
180/180 [==============================] - 9s 48ms/step - loss: 0.4317 - accuracy: 0.8214 - val_loss: 0.4747 - val_accuracy: 0.7925
Epoch 4/200
180/180 [==============================] - 9s 48ms/step - loss: 0.4311 - accuracy: 0.8200 - val_loss: 0.4811 - val_accuracy: 0.7972
Epoch 5/200
180/180 [==============================] - 8s 46ms/step - loss: 0.4269 - accuracy: 0.8212 - val_loss: 0.4776 - val_accuracy: 0.7909
Epoch 6/200
180/180 [==============================] - 8s 47ms/step - loss: 0.4295 - accuracy: 0.8173 - val_loss: 0.4774 - val_accuracy: 0.7956
Epoch 7/200
180/180 [==============================] - 8s 47ms/step - loss: 0.4272 - accuracy: 0.8221 - val_loss: 0.4762 - val_accuracy: 0.8019
Epoch 8/200
180/180 [==============================] - 9s 47ms/step - loss: 0.4302 - accuracy: 0.8201 - val_loss: 0.4852 - val_accuracy: 0.7925
Epoc

In [9]:
np.random.seed(0)
tf.random.set_seed(1)  # for reproducibility
# reading model

model = build_model()


BATCH_SIZE = 32
EPOCHS = 200

# # Cross-validation
n = 10
k_fold = KFold(n_splits=n, shuffle=True, random_state=42)

all_performance = []
tprs = []
mean_fpr = np.linspace(0, 1, 100)
for fold_count, (train_index, val_index) in enumerate(k_fold.split(train)):
    print('*' * 30 + ' the ' + str(fold_count + 1) + ' fold ' + '*' * 30)
    trains, val = train[train_index], train[val_index]
    trains_label, val_label = train_label[train_index], train_label[val_index]
    zaoting = EarlyStopping(monitor='val_loss', patience=13, mode='auto')
    xuexilv = WarmupExponentialDecay(lr_base=0.0002,decay=0.00002,warmup_epochs=2)
    callback_lists=[xuexilv,zaoting]
    model.fit(x=trains, y=trains_label, validation_data=(val, val_label), epochs=EPOCHS,
                batch_size=BATCH_SIZE, shuffle=True,
                callbacks=callback_lists,
                verbose=1)
     # 保存模型

    model.save('./warmup_embedding/model_fold' + str(fold_count+1) + '.h5')

    del model

    model = load_model('./warmup_embedding/model_fold' + str(fold_count+1) + '.h5')

    val_pred = model.predict(val, verbose=1)

    # Sn, Sp, Acc, MCC, AUC
    Sn, Sp, Acc, MCC = show_performance(val_label[:, 1], val_pred[:, 1])
    AUC = roc_auc_score(val_label[:, 1], val_pred[:, 1])
    print('Sn = %f, Sp = %f, Acc = %f, MCC = %f, AUC = %f' % (Sn, Sp, Acc, MCC, AUC))

    performance = [Sn, Sp, Acc, MCC, AUC]
    all_performance.append(performance)
    
all_performance = np.array(all_performance)
print('10 fold result:', all_performance)
performance_mean = performance_mean(all_performance)

model.fit(x=train, y=train_label, validation_data=(test, test_label), epochs=EPOCHS,
                      batch_size=BATCH_SIZE, shuffle=True,
                      callbacks=[EarlyStopping(monitor='val_loss', patience=20, mode='auto')],
                      verbose=1)
model.save('./warmup_embedding/model_test.h5')

del model

model = load_model('./warmup_embedding/model_test.h5')

test_score = model.predict(test)


# Sn, Sp, Acc, MCC, AUC
Sn, Sp, Acc, MCC = show_performance(test_label[:,1], test_score[:,1])
AUC = roc_auc_score(test_label[:,1], test_score[:,1])

print('-----------------------------------------------test---------------------------------------')
print('Sn = %f, Sp = %f, Acc = %f, MCC = %f, AUC = %f' % (Sn, Sp, Acc, MCC, AUC))

******************************Warmup+embedding_64******************************
(None, 15, 64)


2024-06-05 15:12:05.962735: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4226 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:65:00.0, compute capability: 8.6


(None, 15, 48)
****************************** the 1 fold ******************************
Epoch 1/200


2024-06-05 15:12:14.089159: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8800
Could not load symbol cublasGetSmCountTarget from libcublas.so.11. Error: /usr/local/cuda/targets/x86_64-linux/lib/libcublas.so.11: undefined symbol: cublasGetSmCountTarget
2024-06-05 15:12:14.349178: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-06-05 15:12:14.388630: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f2612ddaa10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-06-05 15:12:14.388649: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-06-05 15:12:14.391984: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to e

  1/162 [..............................] - ETA: 21:51 - loss: 3.2299 - accuracy: 0.4062

...

162/162 [==============================] - 9s 54ms/step - loss: 0.7304 - accuracy: 0.6966 - val_loss: 0.7347 - val_accuracy: 0.7108
Epoch 19/200
162/162 [==============================] - 9s 53ms/step - loss: 0.7145 - accuracy: 0.7020 - val_loss: 0.7192 - val_accuracy: 0.6986
Epoch 20/200
162/162 [==============================] - 9s 54ms/step - loss: 0.6989 - accuracy: 0.7067 - val_loss: 0.7024 - val_accuracy: 0.7021
Epoch 21/200
162/162 [==============================] - 9s 53ms/step - loss: 0.6837 - accuracy: 0.7096 - val_loss: 0.6911 - val_accuracy: 0.7091
Epoch 22/200
162/162 [==============================] - 9s 53ms/step - loss: 0.6705 - accuracy: 0.7109 - val_loss: 0.6790 - val_accuracy: 0.7160
Epoch 23/200
162/162 [==============================] - 9s 53ms/step - loss: 0.6586 - accuracy: 0.7121 - val_loss: 0.6766 - val_accuracy: 0.6986
Epoch 24/200
162/162 [==============================] - 9s 54ms/step - loss: 0.6463 - accuracy: 0.7181 - val_loss: 0.6618 - val_accuracy: 0.717

2024-06-05 15:17:10.093696: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.



162/162 [==============================] - 9s 55ms/step - loss: 0.5622 - accuracy: 0.7423 - val_loss: 0.5856 - val_accuracy: 0.7195
Epoch 35/200
162/162 [==============================] - 9s 53ms/step - loss: 0.5552 - accuracy: 0.7445 - val_loss: 0.5734 - val_accuracy: 0.7474
Epoch 36/200
162/162 [==============================] - 9s 53ms/step - loss: 0.5519 - accuracy: 0.7476 - val_loss: 0.5748 - val_accuracy: 0.7352
Epoch 37/200
162/162 [==============================] - 9s 53ms/step - loss: 0.5464 - accuracy: 0.7495 - val_loss: 0.5825 - val_accuracy: 0.7247
Epoch 38/200
162/162 [==============================] - 9s 53ms/step - loss: 0.5424 - accuracy: 0.7493 - val_loss: 0.5679 - val_accuracy: 0.7456
Epoch 39/200
162/162 [==============================] - 9s 54ms/step - loss: 0.5408 - accuracy: 0.7520 - val_loss: 0.5645 - val_accuracy: 0.7422
Epoch 40/200
162/162 [==============================] - 9s 53ms/step - loss: 0.5366 - accuracy: 0.7542 - val_loss: 0.5609 - val_accuracy: 0.740

2024-06-05 15:50:11.179104: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.



180/180 [==============================] - 8s 46ms/step - loss: 0.4410 - accuracy: 0.8109 - val_loss: 0.4768 - val_accuracy: 0.8082
Epoch 3/200
180/180 [==============================] - 9s 48ms/step - loss: 0.4449 - accuracy: 0.8102 - val_loss: 0.4770 - val_accuracy: 0.8003
Epoch 4/200
180/180 [==============================] - 9s 48ms/step - loss: 0.4428 - accuracy: 0.8095 - val_loss: 0.4826 - val_accuracy: 0.8019
Epoch 5/200
180/180 [==============================] - 8s 47ms/step - loss: 0.4418 - accuracy: 0.8137 - val_loss: 0.4743 - val_accuracy: 0.8050
Epoch 6/200
180/180 [==============================] - 8s 47ms/step - loss: 0.4421 - accuracy: 0.8118 - val_loss: 0.4764 - val_accuracy: 0.8082
Epoch 7/200
180/180 [==============================] - 8s 46ms/step - loss: 0.4412 - accuracy: 0.8109 - val_loss: 0.4738 - val_accuracy: 0.8019
Epoch 8/200
180/180 [==============================] - 9s 47ms/step - loss: 0.4424 - accuracy: 0.8114 - val_loss: 0.4852 - val_accuracy: 0.7940
Epoc

In [9]:
np.random.seed(0)
tf.random.set_seed(1)  # for reproducibility
# reading model

model = build_model()


BATCH_SIZE = 32
EPOCHS = 200

# # Cross-validation
n = 10
k_fold = KFold(n_splits=n, shuffle=True, random_state=42)

all_performance = []
tprs = []
mean_fpr = np.linspace(0, 1, 100)
for fold_count, (train_index, val_index) in enumerate(k_fold.split(train)):
    print('*' * 30 + ' the ' + str(fold_count + 1) + ' fold ' + '*' * 30)
    trains, val = train[train_index], train[val_index]
    trains_label, val_label = train_label[train_index], train_label[val_index]
    zaoting = EarlyStopping(monitor='val_loss', patience=13, mode='auto')
    xuexilv = WarmupExponentialDecay(lr_base=0.0002,decay=0.00002,warmup_epochs=2)
    callback_lists=[xuexilv,zaoting]
    model.fit(x=trains, y=trains_label, validation_data=(val, val_label), epochs=EPOCHS,
                batch_size=BATCH_SIZE, shuffle=True,
                callbacks=callback_lists,
                verbose=1)
     # 保存模型

    model.save('./warmup_embedding/model_fold' + str(fold_count+1) + '.h5')

    del model

    model = load_model('./warmup_embedding/model_fold' + str(fold_count+1) + '.h5')

    val_pred = model.predict(val, verbose=1)

    # Sn, Sp, Acc, MCC, AUC
    Sn, Sp, Acc, MCC = show_performance(val_label[:, 1], val_pred[:, 1])
    AUC = roc_auc_score(val_label[:, 1], val_pred[:, 1])
    print('Sn = %f, Sp = %f, Acc = %f, MCC = %f, AUC = %f' % (Sn, Sp, Acc, MCC, AUC))

    performance = [Sn, Sp, Acc, MCC, AUC]
    all_performance.append(performance)
    
all_performance = np.array(all_performance)
print('10 fold result:', all_performance)
performance_mean = performance_mean(all_performance)

model.fit(x=train, y=train_label, validation_data=(test, test_label), epochs=EPOCHS,
                      batch_size=BATCH_SIZE, shuffle=True,
                      callbacks=[EarlyStopping(monitor='val_loss', patience=20, mode='auto')],
                      verbose=1)
model.save('./warmup_embedding/model_test.h5')

del model

model = load_model('./warmup_embedding/model_test.h5')

test_score = model.predict(test)


# Sn, Sp, Acc, MCC, AUC
Sn, Sp, Acc, MCC = show_performance(test_label[:,1], test_score[:,1])
AUC = roc_auc_score(test_label[:,1], test_score[:,1])

print('-----------------------------------------------test---------------------------------------')
print('Sn = %f, Sp = %f, Acc = %f, MCC = %f, AUC = %f' % (Sn, Sp, Acc, MCC, AUC))

******************************Warmup+embedding_32******************************
(None, 15, 32)


2024-06-05 12:09:52.806821: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 8976 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:65:00.0, compute capability: 8.6


(None, 15, 48)
****************************** the 1 fold ******************************
Epoch 1/200


2024-06-05 12:09:59.849609: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8800
Could not load symbol cublasGetSmCountTarget from libcublas.so.11. Error: /usr/local/cuda/targets/x86_64-linux/lib/libcublas.so.11: undefined symbol: cublasGetSmCountTarget
2024-06-05 12:10:00.092845: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-06-05 12:10:00.119126: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f659baa64d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-06-05 12:10:00.119147: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-06-05 12:10:00.122437: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to e

  1/162 [..............................] - ETA: 18:48 - loss: 3.2193 - accuracy: 0.5000

...

162/162 [==============================] - 6s 37ms/step - loss: 0.7508 - accuracy: 0.6733 - val_loss: 0.7510 - val_accuracy: 0.6899
Epoch 19/200
162/162 [==============================] - 6s 36ms/step - loss: 0.7363 - accuracy: 0.6762 - val_loss: 0.7381 - val_accuracy: 0.6742
Epoch 20/200
162/162 [==============================] - 6s 35ms/step - loss: 0.7226 - accuracy: 0.6807 - val_loss: 0.7265 - val_accuracy: 0.6899
Epoch 21/200
162/162 [==============================] - 6s 36ms/step - loss: 0.7081 - accuracy: 0.6782 - val_loss: 0.7140 - val_accuracy: 0.6934
Epoch 22/200
162/162 [==============================] - 6s 36ms/step - loss: 0.6960 - accuracy: 0.6842 - val_loss: 0.6985 - val_accuracy: 0.6899
Epoch 23/200
162/162 [==============================] - 6s 36ms/step - loss: 0.6852 - accuracy: 0.6853 - val_loss: 0.6937 - val_accuracy: 0.6916
Epoch 24/200
162/162 [==============================] - 6s 36ms/step - loss: 0.6740 - accuracy: 0.6914 - val_loss: 0.6796 - val_accuracy: 0.691

2024-06-05 12:13:19.109746: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.



162/162 [==============================] - 6s 36ms/step - loss: 0.5946 - accuracy: 0.7117 - val_loss: 0.6059 - val_accuracy: 0.7021
Epoch 35/200
162/162 [==============================] - 6s 35ms/step - loss: 0.5868 - accuracy: 0.7168 - val_loss: 0.5993 - val_accuracy: 0.7073
Epoch 36/200
162/162 [==============================] - 6s 36ms/step - loss: 0.5814 - accuracy: 0.7168 - val_loss: 0.6014 - val_accuracy: 0.7160
Epoch 37/200
162/162 [==============================] - 6s 36ms/step - loss: 0.5758 - accuracy: 0.7255 - val_loss: 0.6164 - val_accuracy: 0.6899
Epoch 38/200
162/162 [==============================] - 6s 36ms/step - loss: 0.5702 - accuracy: 0.7299 - val_loss: 0.5957 - val_accuracy: 0.7143
Epoch 39/200
162/162 [==============================] - 6s 36ms/step - loss: 0.5650 - accuracy: 0.7309 - val_loss: 0.5916 - val_accuracy: 0.7265
Epoch 40/200
162/162 [==============================] - 6s 35ms/step - loss: 0.5582 - accuracy: 0.7385 - val_loss: 0.5871 - val_accuracy: 0.724

2024-06-05 12:31:37.125971: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.



180/180 [==============================] - 4s 22ms/step - loss: 0.4604 - accuracy: 0.7973 - val_loss: 0.4846 - val_accuracy: 0.7830
Epoch 3/200
180/180 [==============================] - 4s 23ms/step - loss: 0.4630 - accuracy: 0.7922 - val_loss: 0.4929 - val_accuracy: 0.7736
Epoch 4/200
180/180 [==============================] - 4s 22ms/step - loss: 0.4605 - accuracy: 0.7961 - val_loss: 0.4953 - val_accuracy: 0.7736
Epoch 5/200
180/180 [==============================] - 4s 22ms/step - loss: 0.4590 - accuracy: 0.7959 - val_loss: 0.4843 - val_accuracy: 0.7925
Epoch 6/200
180/180 [==============================] - 4s 22ms/step - loss: 0.4573 - accuracy: 0.7962 - val_loss: 0.4881 - val_accuracy: 0.7814
Epoch 7/200
180/180 [==============================] - 4s 22ms/step - loss: 0.4589 - accuracy: 0.7971 - val_loss: 0.4827 - val_accuracy: 0.7862
Epoch 8/200
180/180 [==============================] - 4s 21ms/step - loss: 0.4599 - accuracy: 0.7936 - val_loss: 0.4957 - val_accuracy: 0.7720
Epoc

In [9]:
np.random.seed(0)
tf.random.set_seed(1)  # for reproducibility
# reading model

model = build_model()


BATCH_SIZE = 32
EPOCHS = 200

# # Cross-validation
n = 10
k_fold = KFold(n_splits=n, shuffle=True, random_state=42)

all_performance = []
tprs = []
mean_fpr = np.linspace(0, 1, 100)
for fold_count, (train_index, val_index) in enumerate(k_fold.split(train)):
    print('*' * 30 + ' the ' + str(fold_count + 1) + ' fold ' + '*' * 30)
    trains, val = train[train_index], train[val_index]
    trains_label, val_label = train_label[train_index], train_label[val_index]
    zaoting = EarlyStopping(monitor='val_loss', patience=13, mode='auto')
    xuexilv = WarmupExponentialDecay(lr_base=0.0002,decay=0.00002,warmup_epochs=2)
    callback_lists=[xuexilv,zaoting]
    model.fit(x=trains, y=trains_label, validation_data=(val, val_label), epochs=EPOCHS,
                batch_size=BATCH_SIZE, shuffle=True,
                callbacks=callback_lists,
                verbose=1)
     # 保存模型

    model.save('./warmup_embedding/model_fold' + str(fold_count+1) + '.h5')

    del model

    model = load_model('./warmup_embedding/model_fold' + str(fold_count+1) + '.h5')

    val_pred = model.predict(val, verbose=1)

    # Sn, Sp, Acc, MCC, AUC
    Sn, Sp, Acc, MCC = show_performance(val_label[:, 1], val_pred[:, 1])
    AUC = roc_auc_score(val_label[:, 1], val_pred[:, 1])
    print('Sn = %f, Sp = %f, Acc = %f, MCC = %f, AUC = %f' % (Sn, Sp, Acc, MCC, AUC))

    performance = [Sn, Sp, Acc, MCC, AUC]
    all_performance.append(performance)
    
all_performance = np.array(all_performance)
print('10 fold result:', all_performance)
performance_mean = performance_mean(all_performance)

model.fit(x=train, y=train_label, validation_data=(test, test_label), epochs=EPOCHS,
                      batch_size=BATCH_SIZE, shuffle=True,
                      callbacks=[EarlyStopping(monitor='val_loss', patience=20, mode='auto')],
                      verbose=1)
model.save('./warmup_embedding/model_test.h5')

del model

model = load_model('./warmup_embedding/model_test.h5')

test_score = model.predict(test)


# Sn, Sp, Acc, MCC, AUC
Sn, Sp, Acc, MCC = show_performance(test_label[:,1], test_score[:,1])
AUC = roc_auc_score(test_label[:,1], test_score[:,1])

print('-----------------------------------------------test---------------------------------------')
print('Sn = %f, Sp = %f, Acc = %f, MCC = %f, AUC = %f' % (Sn, Sp, Acc, MCC, AUC))

******************************Warmup+embedding_16******************************
(None, 15, 16)


2024-06-05 10:24:29.135102: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 8164 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:65:00.0, compute capability: 8.6


(None, 15, 48)
****************************** the 1 fold ******************************
Epoch 1/200


2024-06-05 10:24:36.116406: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8800
Could not load symbol cublasGetSmCountTarget from libcublas.so.11. Error: /usr/local/cuda/targets/x86_64-linux/lib/libcublas.so.11: undefined symbol: cublasGetSmCountTarget
2024-06-05 10:24:36.345926: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-06-05 10:24:36.372656: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f4c7c8c4fb0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-06-05 10:24:36.372690: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-06-05 10:24:36.379068: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to e

  1/162 [..............................] - ETA: 18:41 - loss: 3.2083 - accuracy: 0.5000

...

162/162 [==============================] - 6s 37ms/step - loss: 0.7435 - accuracy: 0.6846 - val_loss: 0.7477 - val_accuracy: 0.6777
Epoch 19/200
162/162 [==============================] - 6s 36ms/step - loss: 0.7293 - accuracy: 0.6846 - val_loss: 0.7343 - val_accuracy: 0.6847
Epoch 20/200
162/162 [==============================] - 6s 36ms/step - loss: 0.7158 - accuracy: 0.6902 - val_loss: 0.7237 - val_accuracy: 0.6864
Epoch 21/200
162/162 [==============================] - 6s 36ms/step - loss: 0.7033 - accuracy: 0.6896 - val_loss: 0.7112 - val_accuracy: 0.6760
Epoch 22/200
162/162 [==============================] - 6s 36ms/step - loss: 0.6923 - accuracy: 0.6884 - val_loss: 0.6962 - val_accuracy: 0.6829
Epoch 23/200
162/162 [==============================] - 6s 36ms/step - loss: 0.6820 - accuracy: 0.6921 - val_loss: 0.6903 - val_accuracy: 0.6864
Epoch 24/200
162/162 [==============================] - 6s 36ms/step - loss: 0.6706 - accuracy: 0.6908 - val_loss: 0.6784 - val_accuracy: 0.691

2024-06-05 10:27:55.525742: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.



162/162 [==============================] - 6s 37ms/step - loss: 0.6030 - accuracy: 0.7041 - val_loss: 0.6100 - val_accuracy: 0.7143
Epoch 35/200
162/162 [==============================] - 6s 36ms/step - loss: 0.5969 - accuracy: 0.7086 - val_loss: 0.6063 - val_accuracy: 0.7195
Epoch 36/200
162/162 [==============================] - 6s 35ms/step - loss: 0.5941 - accuracy: 0.7051 - val_loss: 0.6034 - val_accuracy: 0.7125
Epoch 37/200
162/162 [==============================] - 6s 36ms/step - loss: 0.5897 - accuracy: 0.7117 - val_loss: 0.6059 - val_accuracy: 0.7125
Epoch 38/200
162/162 [==============================] - 6s 36ms/step - loss: 0.5861 - accuracy: 0.7094 - val_loss: 0.5966 - val_accuracy: 0.7178
Epoch 39/200
162/162 [==============================] - 6s 36ms/step - loss: 0.5829 - accuracy: 0.7140 - val_loss: 0.5962 - val_accuracy: 0.7195
Epoch 40/200
162/162 [==============================] - 6s 36ms/step - loss: 0.5788 - accuracy: 0.7105 - val_loss: 0.5895 - val_accuracy: 0.724

2024-06-05 10:46:00.173481: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.



180/180 [==============================] - 6s 33ms/step - loss: 0.4612 - accuracy: 0.7934 - val_loss: 0.4798 - val_accuracy: 0.7940
Epoch 3/200
180/180 [==============================] - 6s 33ms/step - loss: 0.4611 - accuracy: 0.7966 - val_loss: 0.4855 - val_accuracy: 0.7909
Epoch 4/200
180/180 [==============================] - 6s 32ms/step - loss: 0.4595 - accuracy: 0.8018 - val_loss: 0.4770 - val_accuracy: 0.7877
Epoch 5/200
180/180 [==============================] - 6s 32ms/step - loss: 0.4613 - accuracy: 0.7976 - val_loss: 0.4774 - val_accuracy: 0.7940
Epoch 6/200
180/180 [==============================] - 6s 32ms/step - loss: 0.4592 - accuracy: 0.8018 - val_loss: 0.4819 - val_accuracy: 0.7877
Epoch 7/200
180/180 [==============================] - 6s 31ms/step - loss: 0.4597 - accuracy: 0.7985 - val_loss: 0.4765 - val_accuracy: 0.7940
Epoch 8/200
180/180 [==============================] - 6s 32ms/step - loss: 0.4602 - accuracy: 0.7936 - val_loss: 0.4813 - val_accuracy: 0.7893
Epoc

In [9]:
np.random.seed(0)
tf.random.set_seed(1)  # for reproducibility
# reading model

model = build_model()


BATCH_SIZE = 32
EPOCHS = 200

# # Cross-validation
n = 10
k_fold = KFold(n_splits=n, shuffle=True, random_state=42)

all_performance = []
tprs = []
mean_fpr = np.linspace(0, 1, 100)
for fold_count, (train_index, val_index) in enumerate(k_fold.split(train)):
    print('*' * 30 + ' the ' + str(fold_count + 1) + ' fold ' + '*' * 30)
    trains, val = train[train_index], train[val_index]
    trains_label, val_label = train_label[train_index], train_label[val_index]
    zaoting = EarlyStopping(monitor='val_loss', patience=13, mode='auto')
    xuexilv = WarmupExponentialDecay(lr_base=0.0002,decay=0.00002,warmup_epochs=2)
    callback_lists=[xuexilv,zaoting]
    model.fit(x=trains, y=trains_label, validation_data=(val, val_label), epochs=EPOCHS,
                batch_size=BATCH_SIZE, shuffle=True,
                callbacks=callback_lists,
                verbose=1)
     # 保存模型

    model.save('./warmup_embedding/model_fold' + str(fold_count+1) + '.h5')

    del model

    model = load_model('./warmup_embedding/model_fold' + str(fold_count+1) + '.h5')

    val_pred = model.predict(val, verbose=1)

    # Sn, Sp, Acc, MCC, AUC
    Sn, Sp, Acc, MCC = show_performance(val_label[:, 1], val_pred[:, 1])
    AUC = roc_auc_score(val_label[:, 1], val_pred[:, 1])
    print('Sn = %f, Sp = %f, Acc = %f, MCC = %f, AUC = %f' % (Sn, Sp, Acc, MCC, AUC))

    performance = [Sn, Sp, Acc, MCC, AUC]
    all_performance.append(performance)
    
all_performance = np.array(all_performance)
print('10 fold result:', all_performance)
performance_mean = performance_mean(all_performance)

model.fit(x=train, y=train_label, validation_data=(test, test_label), epochs=EPOCHS,
                      batch_size=BATCH_SIZE, shuffle=True,
                      callbacks=[EarlyStopping(monitor='val_loss', patience=20, mode='auto')],
                      verbose=1)
model.save('./warmup_embedding/model_test.h5')

del model

model = load_model('./warmup_embedding/model_test.h5')

test_score = model.predict(test)


# Sn, Sp, Acc, MCC, AUC
Sn, Sp, Acc, MCC = show_performance(test_label[:,1], test_score[:,1])
AUC = roc_auc_score(test_label[:,1], test_score[:,1])

print('-----------------------------------------------test---------------------------------------')
print('Sn = %f, Sp = %f, Acc = %f, MCC = %f, AUC = %f' % (Sn, Sp, Acc, MCC, AUC))

******************************Warmup+embedding_8******************************
(None, 15, 8)


2024-06-05 09:48:13.008298: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 8164 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:65:00.0, compute capability: 8.6


(None, 15, 48)
****************************** the 1 fold ******************************
Epoch 1/200


2024-06-05 09:48:20.231614: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8800
Could not load symbol cublasGetSmCountTarget from libcublas.so.11. Error: /usr/local/cuda/targets/x86_64-linux/lib/libcublas.so.11: undefined symbol: cublasGetSmCountTarget
2024-06-05 09:48:20.564157: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-06-05 09:48:20.593763: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55cacf9fd6b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-06-05 09:48:20.593795: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-06-05 09:48:20.610138: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to e

  1/162 [..............................] - ETA: 19:54 - loss: 3.1995 - accuracy: 0.4688

...

162/162 [==============================] - 6s 38ms/step - loss: 0.7537 - accuracy: 0.6706 - val_loss: 0.7600 - val_accuracy: 0.6760
Epoch 19/200
162/162 [==============================] - 6s 36ms/step - loss: 0.7415 - accuracy: 0.6696 - val_loss: 0.7473 - val_accuracy: 0.6760
Epoch 20/200
162/162 [==============================] - 6s 36ms/step - loss: 0.7291 - accuracy: 0.6714 - val_loss: 0.7390 - val_accuracy: 0.6760
Epoch 21/200
162/162 [==============================] - 6s 36ms/step - loss: 0.7171 - accuracy: 0.6737 - val_loss: 0.7271 - val_accuracy: 0.6725
Epoch 22/200
162/162 [==============================] - 6s 36ms/step - loss: 0.7079 - accuracy: 0.6720 - val_loss: 0.7129 - val_accuracy: 0.6829
Epoch 23/200
162/162 [==============================] - 6s 36ms/step - loss: 0.6982 - accuracy: 0.6747 - val_loss: 0.7074 - val_accuracy: 0.6742
Epoch 24/200
162/162 [==============================] - 6s 36ms/step - loss: 0.6878 - accuracy: 0.6809 - val_loss: 0.6966 - val_accuracy: 0.679

2024-06-05 09:51:39.778706: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.



162/162 [==============================] - 6s 37ms/step - loss: 0.6218 - accuracy: 0.6892 - val_loss: 0.6336 - val_accuracy: 0.6829
Epoch 35/200
162/162 [==============================] - 6s 36ms/step - loss: 0.6184 - accuracy: 0.6900 - val_loss: 0.6269 - val_accuracy: 0.6864
Epoch 36/200
162/162 [==============================] - 6s 36ms/step - loss: 0.6142 - accuracy: 0.6863 - val_loss: 0.6259 - val_accuracy: 0.6882
Epoch 37/200
162/162 [==============================] - 6s 36ms/step - loss: 0.6105 - accuracy: 0.6884 - val_loss: 0.6217 - val_accuracy: 0.6916
Epoch 38/200
162/162 [==============================] - 6s 36ms/step - loss: 0.6079 - accuracy: 0.6908 - val_loss: 0.6205 - val_accuracy: 0.6864
Epoch 39/200
162/162 [==============================] - 6s 36ms/step - loss: 0.6051 - accuracy: 0.6900 - val_loss: 0.6190 - val_accuracy: 0.6777
Epoch 40/200
162/162 [==============================] - 6s 36ms/step - loss: 0.6013 - accuracy: 0.6925 - val_loss: 0.6119 - val_accuracy: 0.691

2024-06-05 10:14:15.452535: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.



180/180 [==============================] - 6s 33ms/step - loss: 0.4793 - accuracy: 0.7828 - val_loss: 0.4732 - val_accuracy: 0.7925
Epoch 3/200
180/180 [==============================] - 6s 33ms/step - loss: 0.4792 - accuracy: 0.7828 - val_loss: 0.4799 - val_accuracy: 0.7846
Epoch 4/200
180/180 [==============================] - 6s 32ms/step - loss: 0.4777 - accuracy: 0.7828 - val_loss: 0.4755 - val_accuracy: 0.7799
Epoch 5/200
180/180 [==============================] - 6s 32ms/step - loss: 0.4777 - accuracy: 0.7840 - val_loss: 0.4724 - val_accuracy: 0.7925
Epoch 6/200
180/180 [==============================] - 6s 32ms/step - loss: 0.4780 - accuracy: 0.7889 - val_loss: 0.4723 - val_accuracy: 0.7862
Epoch 7/200
180/180 [==============================] - 6s 32ms/step - loss: 0.4793 - accuracy: 0.7821 - val_loss: 0.4727 - val_accuracy: 0.7940
Epoch 8/200
180/180 [==============================] - 6s 32ms/step - loss: 0.4780 - accuracy: 0.7840 - val_loss: 0.4755 - val_accuracy: 0.7940
Epoc